# Process for creating keywords

1. Put your data in a tsv, col 1 is an index, col 3 is raw text
2. You should have a reference set, which includes the target.  So you need two files
* target: A small set of documents which are representatives of what we are looking for
* source:  A larger set of documents which include the background from where we are filtering
3. Run them through the notebook in the notebooks directory.  The output of this at the bottom cell should be a list of lists, containing the words of the keyword filter.
4. Cut and paste those lists into this notebook below, the output of which should be a file called language_key_map.wdc and put it in the word2vec folder.  This file represents the coiled style retina.
5. Put a copy of the raw data into finterprint_viewer/retina
6. Change the references in fingerprint_viewer/app.py to point to hte wdc file and the source of the data.
7. That should be it, run with `make run`

## Notes
The data should be cleaned the same way, this is a TODO to merge later but for now, make sure the cleaning function in the first notebook, matches the one in the app.py file to clean the lines.  Otherwise you won't get matches on the output words because they may be capitalized differently or may have some puncutation left etc.

There's too many TODOs to put here, but allthe parts are there, just need to automate process and clean up code before more features, or UI gets updated.

In [1]:
!pwd

/Users/efrain/niarfe/fprint/fingerprint_viewer


In [2]:
some_word_list = ['zero','uno','dos','tres','quatro','cinco','seis','siete','ocho','nueve','diez','onze','doce','trece','catorze','one','two','three','four','five','six','seven','eight','nine','ten']

# define grid directions
## top left corner centered
* top left is 0,0 (x,y)
* right movement is x+
* left movement is x-
* up movement is y-
* down movement is y+

In [3]:
# Assuming we are passed 'center' of 4x4 grid, whose center is 1,1

xinit = 1
yinit = 1

wordmatrix = [
    ['','','','',''],
    ['','','','',''],
    ['','','','',''],
    ['','','','',''],
    ['','','','',''],    
]

def curl_list_around_xy(x, y, box_size, wordmatrix, wordlist):
    direction = 1
    iteration = 1
    coordinates = []
    xdir = -1
    ydir = -1

    iterl = 0
    print x,y
    wordmatrix[x][y] = wordlist[iterl]
    iterl += 1
    for iteration in range(1,box_size+1):
        for idy in range(1,iteration+1):
            y = y + 1*ydir
            print "y",iteration, x, y
            wordmatrix[x][y] = wordlist[iterl]
            iterl += 1
        ydir *= -1
        for idx in range(1,iteration+1):
            x = x + 1*xdir
            print "x",iteration, x, y
            wordmatrix[x][y] = wordlist[iterl]
            iterl += 1
        xdir *= -1
        if x >= 4 or y >= 4:
            break
    return wordmatrix


    


curl_list_around_xy(2,2,4,wordmatrix, some_word_list)
for line in wordmatrix:
    print line

2 2
y 1 2 1
x 1 1 1
y 2 1 2
y 2 1 3
x 2 2 3
x 2 3 3
y 3 3 2
y 3 3 1
y 3 3 0
x 3 2 0
x 3 1 0
x 3 0 0
y 4 0 1
y 4 0 2
y 4 0 3
y 4 0 4
x 4 1 4
x 4 2 4
x 4 3 4
x 4 4 4
['doce', 'trece', 'catorze', 'one', 'two']
['onze', 'dos', 'tres', 'quatro', 'three']
['diez', 'uno', 'zero', 'cinco', 'four']
['nueve', 'ocho', 'siete', 'seis', 'five']
['', '', '', '', 'six']


In [4]:
def coil_to_box(x_top_right, y_top_right, box_size, wordlist):
    """
    x_top_right: int, x location of top right subbox in larger matrix
    y_top_right: int, y location of top right subbox in larger matrix
    box_size: int, number of dots/pixels for box
    wordlist: list<strings>, list of words, assuming in order of priority from 0 to length.
    ** Assumptions
    - box is square
    - words will coil from center and circle out
    - words will be taken from list to fill box, remainder will be ignored, and missing will be blank
    return: list<string> where each string is "word (x,y)"
    """
    direction = 1
    iteration = 1
    coordinates = []
    xdir = -1
    ydir = -1

    iterl = 0
    print x,y
    wordmatrix[x][y] = wordlist[iterl]
    iterl += 1
    for iteration in range(1,box_size+1):
        for idy in range(1,iteration+1):
            y = y + 1*ydir
            print "y",iteration, x, y
            wordmatrix[x][y] = wordlist[iterl]
            iterl += 1
        ydir *= -1
        for idx in range(1,iteration+1):
            x = x + 1*xdir
            print "x",iteration, x, y
            wordmatrix[x][y] = wordlist[iterl]
            iterl += 1
        xdir *= -1
        if x >= 4 or y >= 4:
            break
    return wordmatrix

In [5]:

class Swirl:
    def __init__(self, x_init, y_init):
        self.x = x_init
        self.y = y_init
        self.dir = (0,-1)
        self.iteration_length = 1
        self.iteration = 0
        

    def turn(self):
        if self.dir == (0,-1):
            self.dir = (-1,0)
        elif self.dir == (-1,0):
            self.dir = (0,1)
            self.iteration_length += 1
        elif self.dir == (0,1):
            self.dir = (1,0)
        elif self.dir == (1,0):
            self.dir = (0,-1)
            self.iteration_length += 1
        else:
            raise "Invalid direction"
            
    def step(self):
        retx = self.x
        rety = self.y
        #self.display()
        if self.iteration >= self.iteration_length:
            self.iteration = 0
            self.turn()
            
        if self.dir == (1,0) or self.dir == (-1,0):
            self.iteration += 1
            self.x += self.dir[0]
        else:
            self.iteration += 1
            self.y += self.dir[1]
        return retx, rety


    def display(self):
        print self.x, self.y #, self.dir, self.iteration, self.iteration_length

import math

def grid_it(grid_x, grid_y, box_size):
    """
        grid_x: int, x location of top left corner inside larger grid
        grid_y: int, y location of top left corner inside larger grid
        box_size: int, width of inset box to coil into
    """
    n_pixels = box_size*box_size                   # Total number of pixels, ex: size 16, then 16*16=256 pixels
    n = int(math.ceil(math.sqrt(n_pixels/2))) - 1  # 
    tile_x = box_size*grid_x
    tile_y = box_size*grid_y
    
    print "n:",n
    print "n_pixels:", n_pixels
    print "box_size:", box_size
    m = Swirl(tile_x+n,tile_y+n)
    for i in range(n_pixels-1):
        m.step()

        
grid_it(0,0,16)

print "DONE"

n: 11
n_pixels: 256
box_size: 16
DONE


In [6]:
langs = [   
[
        "--yes 200--", " date : false "," Linguini "," 5 , Value "," false , text "," Served with choice "," . The service "," : 16 . "," Oz . "," Tavern "," text : null "," . Food "," price : 16 "," Good for a "," bar , "," Cup "," null , quote "," of side } "," Happy Hour "," the restaurant "," answer "," and - "," a few "," ambiance "," Pan "," value : 5 "," Shrimp & "," was good "," portions "," Cal } , "," l / 05 "," side } , "," 9 . 95 "," Hand - "," . ] "," . Good "," : 23 "," P "," a Date "," Served with rice "," lettuce , "," waitress "," . ] } "," for a Date "," We had "," waiter "," , shrimp "," , avocado , "," Grill "," 14 . 95 "," sum "," back . "," bars "," 4 , service "," extra "," categories : Bars "," - 10 - "," dirty rice } "," - 08 - "," to eat "," . But "," breakfast "," 08 - "," & Cheese "," chinese "," Sausage "," pork "," l / 10 "," l / 0e "," appetizer "," was not "," , 2005 "," Saturday "," Baked "," Spicy "," 0400 } "," : Red "," 5 , Service "," - 0400 "," 0400 } , "," 0400 "," : Good for "," 00 - 0400 "," - 0400 } "," there . "," linguini "," text : A "," menu . "," Bacon "," double "," , Great for "," ice "," Fun "," 2005 , text "," quote : Good "," 2005 , "," Groups "," evening "," vodka "," oz "," juice "," , 2005 , "," disappointed "," la "," 19 , 2017 "," found "," Style "," enough "," : 17 "," crabmeat "," 24 , "," jumbo "," 27 , "," Crispy "," service and "," date : February "," date : December "," Cheese , prices "," The food is "," place , "," title : , "," party "," Options } , "," blend "," : Hand - "," Nice "," : , rating "," Shrimp , description "," 55 "," Bay "," title : Fried "," for Groups "," Options } "," K "," Good for Groups "," - made "," - pizza "," Pasta "," Comida "," stuffed "," Linguini Alfredo "," user "," of side . "," was excellent "," Excellent "," - and - "," : New "," start_time : 2015 "," photo1jpg . jpg "," Oz . ] "," dim "," Barry ' s "," 3 , categories "," count : 3 "," tavern "," Barry ' "," / photo1jpg . "," Billy "," friend "," Goat "," / photo1jpg "," slow "," , California "," photo1jpg "," Casual "," Friendly "," decor "," a Date , "," quite "," 75 , "," to have "," black "," and choice "," : 25 "," Date , "," and choice of "," said "," this restaurant "," City "," Cocktail "," photo1jpg . "," great , "," / user "," Bar } "," half "," : Love "," loved "," my favorite "," you want "," 10 . 95 "," friendly and "," Friday "," start_time : 2017 "," salad } , "," thin "," 23 , "," Ask "," Eating "," : Pan "," tomatoes , "," , which ",
    ],
    [
        "--no 200--", " Apartment "," default } ] "," association "," - association "," Management } , "," Condominiums , "," Association "," Condominiums } "," condominiums "," Condominiums } , "," : Condominiums } "," Dallas "," categories : Condominiums "," - condominiums "," : Condominiums , "," Condominiums , number_of_ratings "," Association , "," primary_category : Condominiums "," art "," number_of_ratings : 0 "," pubapi_4r5n5vtdr3 , claimed "," - of - "," of - "," owners "," Residential "," condominiums - "," - association / "," association / "," - condominiums - "," Operators "," filled with "," : Residential "," . org "," org "," Operators } "," / Dallas / "," Texas / Dallas "," / Dallas "," Dallas / "," Condo "," false , primary_category "," 31317911 "," - owners "," Building "," apartment "," Real Estate } "," false , tip_count "," Estate } "," Report } , "," Apartments "," Background Report } "," 31434197 "," Owners "," Report } "," Association , type "," Complex "," Condominium Operators } "," Cakes "," Complex } "," Complex } , "," : Condominium Operators "," Condominium Operators "," : Apartment "," - tx "," tx "," Operators } , "," property "," Condominium Complex "," Estate Management } "," 32025769 "," Condominium Complex } "," Estate Management "," Agents "," estate "," : Condominium Complex "," Real Estate Agents "," Estate Agents "," categories : Apartment "," Real Estate Management "," icing "," condominium "," dough "," images / "," living "," / images / "," { websites : "," true } ] "," Museum "," { websites "," apartments "," : Condominium Management "," home - "," management "," Homes "," Condominium Management "," Oteri "," Art "," Condominium Management } "," - place "," Condos "," - home "," tip_count : 0 "," place - "," ] , company_founded "," exhibit "," - place - "," - home - "," 31956120 "," Services } , "," . png "," Wedding "," Oteri ' "," 0 , users_count "," Buildings "," Cake , "," Oteri ' s "," png "," creek - gardens "," / turtle "," owners - "," - creek "," creek - "," creek "," condominium - "," Creek "," jpg } ] "," Gardens "," turtle "," org / "," turtle - creek "," museum "," turtle - "," - gardens "," Cake , description "," . org / "," gardens "," - owners - "," 31498161 "," real estate "," Cakes } "," / turtle - "," - creek - "," - condominium "," - condominium - "," ) , categories_plural "," : Turtle "," - association - "," - gardens - "," categories : Residential "," lafayette "," Creek Gardens "," 117 "," Turtle Creek "," - tx / "," Residential Building "," association - "," gardens - "," owners - association "," . png } "," png } "," tx / "," lafayette - place "," / lafayette "," Turtle Creek Gardens "," Cakes } , "," / lafayette - "," : Turtle Creek "," lafayette - "," ( Apartments "," / Condos ) "," Condos ) } "," Apartment / "," Condo ) "," / Condo ) "," 31317911 . "," : Residential Building "," a variety of "," moist "," { foursquare "," / Condos "," Condos ) "," Buildings ( "," Residential Buildings "," Buildings ( Apartments "," home - owners "," a variety "," { foursquare : "," Building ( "," Building ( Apartment "," Gardens , "," 31317911 . aspx "," Condo ) , "," ( Apartment "," : Residential Buildings "," ( Apartments / "," glaze } "," inc / 31317911 "," Apartments / "," Dallas / turtle "," warwick ",
    ],
]

In [7]:

with open('language_key_map.wdc','w') as target:
    for row in range(4):
        for col in range(4):
            try:
                lang = langs[row*4 + col]
            except:
                continue
            print lang.pop(0)
            m = Swirl(row*16 + 8,col*16 + 8)
            for idx in range(len(lang)):
                x, y = m.step()
                print "{}\t({},{})".format(lang[idx],x,y)
                target.write("{}\t({},{})\n".format(lang[idx],x,y))


--yes 200--
 date : false 	(8,8)
 Linguini 	(8,7)
 5 , Value 	(7,7)
 false , text 	(7,8)
 Served with choice 	(7,9)
 . The service 	(8,9)
 : 16 . 	(9,9)
 Oz . 	(9,8)
 Tavern 	(9,7)
 text : null 	(9,6)
 . Food 	(8,6)
 price : 16 	(7,6)
 Good for a 	(6,6)
 bar , 	(6,7)
 Cup 	(6,8)
 null , quote 	(6,9)
 of side } 	(6,10)
 Happy Hour 	(7,10)
 the restaurant 	(8,10)
 answer 	(9,10)
 and - 	(10,10)
 a few 	(10,9)
 ambiance 	(10,8)
 Pan 	(10,7)
 value : 5 	(10,6)
 Shrimp & 	(10,5)
 was good 	(9,5)
 portions 	(8,5)
 Cal } , 	(7,5)
 l / 05 	(6,5)
 side } , 	(5,5)
 9 . 95 	(5,6)
 Hand - 	(5,7)
 . ] 	(5,8)
 . Good 	(5,9)
 : 23 	(5,10)
 P 	(5,11)
 a Date 	(6,11)
 Served with rice 	(7,11)
 lettuce , 	(8,11)
 waitress 	(9,11)
 . ] } 	(10,11)
 for a Date 	(11,11)
 We had 	(11,10)
 waiter 	(11,9)
 , shrimp 	(11,8)
 , avocado , 	(11,7)
 Grill 	(11,6)
 14 . 95 	(11,5)
 sum 	(11,4)
 back . 	(10,4)
 bars 	(9,4)
 4 , service 	(8,4)
 extra 	(7,4)
 categories : Bars 	(6,4)
 - 10 - 	(5,4)
 dirty rice } 	(4,4)